# Identify PM using features

In [1]:
import os
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf

from sklearn.utils import shuffle, resample
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import model_from_json, Model

In [2]:
# Load the CSV file
df = pd.read_excel(r'../../Datasets/PALM/All/Labels.xlsx')

# Define constants
IMAGE_SIZE = (256, 256)  # Example size, adjust to your models
NUM_CLASSES = 2

In [3]:
# OD
with open(r'../features_segmentation/OD.json', 'r') as json_file:
    model_json = json_file.read()
model_OD = model_from_json(model_json)
model_OD.load_weights(r'../features_segmentation/OD.weights.h5')
model_OD.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Atrophy
with open(r'../features_segmentation/Atrophy.json', 'r') as json_file:
    model_json = json_file.read()
model_atrophy = model_from_json(model_json)
model_atrophy.load_weights(r'../features_segmentation/Atrophy.weights.h5')
model_atrophy.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2024-12-14 11:56:55.792642: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-14 11:56:55.792850: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Define function to preprocess images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=IMAGE_SIZE)
    img = img_to_array(img)
    img = img / 255.0
    return img

def generate_feature_map_OD(image_path):
    # Apply the specific preprocessing method for each model
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    
    mask1 = model_OD.predict(img, verbose=False)  # Predict mask
    mask1 = (mask1 > 0.1).astype(int)  # Convert to binary

    combined = np.transpose(combined, (1, 2, 0, 3))
    combined = np.squeeze(combined)
    
    return combined

def generate_feature_map_ODA(image_path):
    # Apply the specific preprocessing method for each model
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    
    mask1 = model_OD.predict(img, verbose=False)  # Predict mask
    mask1 = (mask1 > 0.1).astype(int)  # Convert to binary
    
    mask2 = model_atrophy.predict(img, verbose=False)  # Predict mask
    mask2 = (mask2 > 0.1).astype(int)  # Convert to binary

    # Combine masks into a single feature map
    combined = np.concatenate([mask1, mask2])  # Shape: (H, W, 4)
    combined = np.transpose(combined, (1, 2, 0, 3))
    combined = np.squeeze(combined)
    
    return combined


## Both OD and Atrophy

In [5]:
# Prepare dataset
dataset_path = r'../../Datasets/PALM/All/Images'

X_features = []
y_labels = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    image_path = row['imgName']
    label = row['Label']
    
    image_path = os.path.join(dataset_path, image_path)
    
    # change function to generate_feature_map_OD if only using OD
    combined_features = generate_feature_map_ODA(image_path)
    X_features.append(combined_features)
    y_labels.append(label)

X_features = np.array(X_features)  # Shape: (num_samples, H, W, 4)
y_labels = tf.keras.utils.to_categorical(y_labels, NUM_CLASSES)

Processing rows:   0%|          | 0/1200 [00:00<?, ?it/s]2024-12-14 11:57:24.452314: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-12-14 11:57:24.521479: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-12-14 11:57:24.820558: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
Processing rows: 100%|██████████| 1200/1200 [02:35<00:00,  7.71it/s]


In [6]:
# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_features, y_labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.8, random_state=42)

print(f"Train set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")

Train set: (840, 256, 256, 2), (840, 2)
Validation set: (72, 256, 256, 2), (72, 2)
Test set: (288, 256, 256, 2), (288, 2)


In [7]:
X_train.shape, X_val.shape, X_test.shape

((840, 256, 256, 2), (72, 256, 256, 2), (288, 256, 256, 2))

In [8]:
y_train.shape, y_val.shape, y_test.shape

((840, 2), (72, 2), (288, 2))

In [9]:
classification_model = tf.keras.Sequential([
    # First Conv2D layer
    # Change input shape to (256, 256, 1) if using only OD
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 2)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),  # Optional pooling layer

    # Second Conv2D layer
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),  # Optional pooling layer

    # Third Conv2D layer
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),

    # Global pooling to reduce to 2D tensor
    tf.keras.layers.GlobalAveragePooling2D(),

    # Fully connected layers
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2, activation='softmax')  # Output layer with 5 classes
])

classification_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classification_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      608       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 global_average_pooling2d (G  (None, 128)              0

In [10]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=20,          # Number of epochs with no improvement before stopping
    restore_best_weights=True  # Restore weights from the epoch with the best validation loss
)

In [11]:
# Train the model
history = classification_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=200,
    callbacks=[early_stopping]
)

Epoch 1/200


2024-12-14 12:00:12.995783: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


27/27 [==============================] - ETA: 0s - loss: 0.6960 - accuracy: 0.5238

2024-12-14 12:00:17.041229: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


27/27 [==============================] - 5s 127ms/step - loss: 0.6960 - accuracy: 0.5238 - val_loss: 0.6947 - val_accuracy: 0.4583
Epoch 2/200
27/27 [==============================] - 2s 85ms/step - loss: 0.6904 - accuracy: 0.5238 - val_loss: 0.6936 - val_accuracy: 0.4583
Epoch 3/200
27/27 [==============================] - 2s 85ms/step - loss: 0.5538 - accuracy: 0.5595 - val_loss: 0.4814 - val_accuracy: 0.9028
Epoch 4/200
27/27 [==============================] - 2s 79ms/step - loss: 0.4361 - accuracy: 0.8488 - val_loss: 0.4572 - val_accuracy: 0.8889
Epoch 5/200
27/27 [==============================] - 2s 89ms/step - loss: 0.3614 - accuracy: 0.8798 - val_loss: 0.3559 - val_accuracy: 0.8472
Epoch 6/200
27/27 [==============================] - 2s 85ms/step - loss: 0.3664 - accuracy: 0.8821 - val_loss: 0.3054 - val_accuracy: 0.9028
Epoch 7/200
27/27 [==============================] - 2s 82ms/step - loss: 0.2995 - accuracy: 0.8929 - val_loss: 0.2833 - val_accuracy: 0.9167
Epoch 8/200
27/27

In [12]:
# Evaluate the model on the val set
val_loss, val_accuracy = classification_model.evaluate(X_val, y_val, verbose=2)
print(f"Val Accuracy: {val_accuracy * 100:.2f}%")
print(f"Val Loss: {val_loss:.4f}")

3/3 - 0s - loss: 0.1672 - accuracy: 0.9306 - 90ms/epoch - 30ms/step
Val Accuracy: 93.06%
Val Loss: 0.1672


In [13]:
# Evaluate the model on the test set
test_loss, test_accuracy = classification_model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

9/9 - 0s - loss: 0.1743 - accuracy: 0.9444 - 306ms/epoch - 34ms/step
Test Accuracy: 94.44%
Test Loss: 0.1743


In [14]:
# Save the model
model_json = classification_model.to_json()
with open("../models_features/CNN_ODA.json", "w") as json_file:
    json_file.write(model_json)

# Saving the model and weights
classification_model.save_weights('../models_features/CNN_ODA.weights.h5')